wilds
ArmorData.user.3.json: armor data


In [3]:
# head cell
import json
import pandas as pd
import numpy as np
import re

with open('mhwilds/ArmorData.user.3.json') as file:
    raw = json.load(file)

app.user_data.ArmorData.cData._Index: Rename to Index
app.user_data.ArmorData.cData._DataValue: DataValue
app.user_data.ArmorData.cData._Series.app.ArmorDef.SERIES_Serializable._Value: [30074]ID_213 -> ID_213: Name
app.user_data.ArmorData.cData._PartsType.app.ArmorDef.ARMOR_PARTS_Serializable._Value: [0]HELM -> HELM
app.user_data.ArmorData.cData._Name: Send to front
app.user_data.ArmorData.cData._Explain: Delete
app.user_data.ArmorData.cData._Defense: Rename to Defense
app.user_data.ArmorData.cData._Resistance: Break into Fire | Water | Thunder | Ice | Dragon
app.user_data.ArmorData.cData._SlotLevel: Break into Slot 1 | Slot 2 | Slot 3
app.user_data.ArmorData.cData._Skill: Break into Skill 1 | Skill 2 | Skill 3
app.user_data.ArmorData.cData._SkillLevel: Break into Skill 1 LV | Skill 2 LV | Skill 3 LV

In [4]:
# Armor
sect = raw.get('app.user_data.ArmorData').get('_Values')
df = pd.json_normalize(sect)

df['Index'] = df['app.user_data.ArmorData.cData._Index']
# df['Data_Value'] = df['app.user_data.ArmorData.cData._DataValue']
df['Series'] = df['app.user_data.ArmorData.cData._Series.app.ArmorDef.SERIES_Serializable._Value'].str.extract(r'\](.*)', expand=False)
df['Part'] = df['app.user_data.ArmorData.cData._PartsType.app.ArmorDef.ARMOR_PARTS_Serializable._Value'].str.extract(r'\](.*)', expand=False)
df['Defense'] = df['app.user_data.ArmorData.cData._Defense']
df[['Fire','Water','Thunder','Ice','Dragon']] = pd.DataFrame(df['app.user_data.ArmorData.cData._Resistance'].tolist(), index=df.index)
df['Slot_1_LV'] = df['app.user_data.ArmorData.cData._SlotLevel'].apply(lambda x: int(re.search(r'\[(\d+)\]', x[0]['app.EquipDef.SlotLevel_Serializable']['_Value']).group(1)))
df['Slot_2_LV'] = df['app.user_data.ArmorData.cData._SlotLevel'].apply(lambda x: int(re.search(r'\[(\d+)\]', x[1]['app.EquipDef.SlotLevel_Serializable']['_Value']).group(1)))
df['Slot_3_LV'] = df['app.user_data.ArmorData.cData._SlotLevel'].apply(lambda x: int(re.search(r'\[(\d+)\]', x[2]['app.EquipDef.SlotLevel_Serializable']['_Value']).group(1)))
df['Set_Bonus'] = df['app.user_data.ArmorData.cData._Skill'].apply(lambda x: x[0]['app.HunterDef.Skill_Serializable']['_Value'])
df['Set_Bonus'] = df['Set_Bonus'].apply(lambda x: x.split(']')[1] if ']' in x else x)
df['Group_Skill'] = df['app.user_data.ArmorData.cData._Skill'].apply(lambda x: x[1]['app.HunterDef.Skill_Serializable']['_Value'])
df['Group_Skill'] = df['Group_Skill'].apply(lambda x: x.split(']')[1] if ']' in x else x)
df['Skill_1'] = df['app.user_data.ArmorData.cData._Skill'].apply(lambda x: x[2]['app.HunterDef.Skill_Serializable']['_Value'])
df['Skill_1'] = df['Skill_1'].apply(lambda x: x.split(']')[1] if ']' in x else x)
df['Skill_2'] = df['app.user_data.ArmorData.cData._Skill'].apply(lambda x: x[3]['app.HunterDef.Skill_Serializable']['_Value'])
df['Skill_2'] = df['Skill_2'].apply(lambda x: x.split(']')[1] if ']' in x else x)
df['Skill_3'] = df['app.user_data.ArmorData.cData._Skill'].apply(lambda x: x[4]['app.HunterDef.Skill_Serializable']['_Value'])
df['Skill_3'] = df['Skill_3'].apply(lambda x: x.split(']')[1] if ']' in x else x)
df['Skill_4'] = df['app.user_data.ArmorData.cData._Skill'].apply(lambda x: x[5]['app.HunterDef.Skill_Serializable']['_Value'])
df['Skill_4'] = df['Skill_4'].apply(lambda x: x.split(']')[1] if ']' in x else x)
df['Skill_5'] = df['app.user_data.ArmorData.cData._Skill'].apply(lambda x: x[6]['app.HunterDef.Skill_Serializable']['_Value'])
df['Skill_5'] = df['Skill_5'].apply(lambda x: x.split(']')[1] if ']' in x else x)
df[['Set_Bonus_LV','Group_Skill_LV','Skill_1_LV','Skill_2_LV','Skill_3_LV','Skill_4_LV','Skill_5_LV']] = pd.DataFrame(df['app.user_data.ArmorData.cData._SkillLevel'].tolist(), index=df.index)

df_armor=df[['Index','Series','Part','Defense','Fire','Water','Thunder','Ice','Dragon','Slot_1_LV','Slot_2_LV','Slot_3_LV',
       'Skill_1','Skill_1_LV','Skill_2','Skill_2_LV','Skill_3','Skill_3_LV','Skill_4','Skill_4_LV','Skill_5','Skill_5_LV','Set_Bonus','Set_Bonus_LV','Group_Skill','Group_Skill_LV']].copy()

df_armor['Skill_1'] = df_armor['Skill_1'].replace(dict_skills)
df_armor['Skill_2'] = df_armor['Skill_2'].replace(dict_skills)
df_armor['Skill_3'] = df_armor['Skill_3'].replace(dict_skills)
df_armor['Skill_4'] = df_armor['Skill_4'].replace(dict_skills)
df_armor['Skill_5'] = df_armor['Skill_5'].replace(dict_skills)
df_armor['Set_Bonus'] = df_armor['Set_Bonus'].replace(dict_skills)
df_armor['Group_Skill'] = df_armor['Group_Skill'].replace(dict_skills)

# df_armor.head()
df_armor.to_excel("Wilds_Armor.xlsx")



amulets
AmuletData.user.3.json
    _Index: 
    _DataID: 
    _Lv: 
    _AmuletType: 
    _SortId: 
    _Rare: 
    _Skill: 
    _SkillLevel: 

AmuletRecipeData.user.3.json

In [72]:
# head cell
import json
import pandas as pd
import numpy as np
import re

with open('mhwilds/AmuletData.user.3.json') as file:
    raw = json.load(file)
sect = raw.get('app.user_data.AmuletData').get('_Values')
df = pd.json_normalize(sect)

In [73]:
df['Skill_1'] = df['app.user_data.AmuletData.cData._Skill'].apply(lambda x: x[0]['app.HunterDef.Skill_Serializable']['_Value'])
df['Skill_1'] = df['Skill_1'].apply(lambda x: x.split(']')[1] if ']' in x else x)
df['Skill_2'] = df['app.user_data.AmuletData.cData._Skill'].apply(lambda x: x[1]['app.HunterDef.Skill_Serializable']['_Value'])
df['Skill_2'] = df['Skill_2'].apply(lambda x: x.split(']')[1] if ']' in x else x)
df['Skill_3'] = df['app.user_data.AmuletData.cData._Skill'].apply(lambda x: x[2]['app.HunterDef.Skill_Serializable']['_Value'])
df['Skill_3'] = df['Skill_3'].apply(lambda x: x.split(']')[1] if ']' in x else x)
df['Skill_1_LV'] = df['app.user_data.AmuletData.cData._SkillLevel'].apply(lambda x: int(x[0]))
df['Skill_2_LV'] = df['app.user_data.AmuletData.cData._SkillLevel'].apply(lambda x: int(x[1]))
df['Skill_3_LV'] = df['app.user_data.AmuletData.cData._SkillLevel'].apply(lambda x: int(x[2]))
df['Index'] = df['app.user_data.AmuletData.cData._Index']
df['Data_ID'] = df['app.user_data.AmuletData.cData._DataId']
df['Amulet_LV'] = df['app.user_data.AmuletData.cData._Lv']
df['Sort_ID'] = df['app.user_data.AmuletData.cData._SortId']
df['Rarity'] = df['app.user_data.AmuletData.cData._Rare.app.ItemDef.RARE_Serializable._Value'].str.extract(r'\](.*)', expand=False)

df_amu = df[['Index','Data_ID','Sort_ID','Rarity','Amulet_LV','Skill_1','Skill_1_LV','Skill_2','Skill_2_LV','Skill_3','Skill_3_LV']].copy()

df_amu['Skill_1'] = df_amu['Skill_1'].replace(dict_skills)
df_amu['Skill_2'] = df_amu['Skill_2'].replace(dict_skills)
df_amu['Skill_3'] = df_amu['Skill_3'].replace(dict_skills)

# df_amu.head()
df_amu.to_excel('Wilds_Amulet.xlsx')

Accessory

In [6]:
# head cell
import json
import pandas as pd
import numpy as np
import re

with open('mhwilds/AccessoryData.user.3.json') as file:
    raw = json.load(file)
sect = raw.get('app.user_data.AccessoryData').get('_Values')
df = pd.json_normalize(sect)

In [7]:
df['Index'] = df['app.user_data.AccessoryData.cData._Index']
df['Acc_ID'] = df['app.user_data.AccessoryData.cData._AccessoryId.app.EquipDef.ACCESSORY_ID_Serializable._Value'].str.extract(r'\](.*)', expand=False)
df['Acc_Type'] = df['app.user_data.AccessoryData.cData._AccessoryType.app.EquipDef.ACCESSORY_TYPE_Serializable._Value'].str.extract(r'\](.*)', expand=False)
df['Sort_ID'] = df['app.user_data.AccessoryData.cData._SortId']
df['Rarity'] = df['app.user_data.AccessoryData.cData._Rare.app.ItemDef.RARE_Serializable._Value'].str.extract(r'\](.*)', expand=False)
df['Size'] = df['app.user_data.AccessoryData.cData._SlotLevelAcc.app.EquipDef.SlotLevel_Serializable._Value'].apply(lambda x: (lambda m: m.group(1) if m else '')(re.search(r'\[(.*?)\]', x)))
df['Skill_1'] = df['app.user_data.AccessoryData.cData._Skill'].apply(lambda x: x[0]['app.HunterDef.Skill_Serializable']['_Value'])
df['Skill_1'] = df['Skill_1'].apply(lambda x: x.split(']')[1] if ']' in x else x)
df['Skill_2'] = df['app.user_data.AccessoryData.cData._Skill'].apply(lambda x: x[1]['app.HunterDef.Skill_Serializable']['_Value'])
df['Skill_2'] = df['Skill_2'].apply(lambda x: x.split(']')[1] if ']' in x else x)
df['Skill_1_LV'] = df['app.user_data.AccessoryData.cData._SkillLevel'].apply(lambda x: int(x[0]))
df['Skill_2_LV'] = df['app.user_data.AccessoryData.cData._SkillLevel'].apply(lambda x: int(x[1]))

df_acc = df[['Index','Acc_ID','Acc_Type','Sort_ID','Rarity','Size','Skill_1','Skill_1_LV','Skill_2','Skill_2_LV']].copy()

df_acc['Skill_1'] = df_acc['Skill_1'].replace(dict_skills)
df_acc['Skill_2'] = df_acc['Skill_2'].replace(dict_skills)

# df_acc.head()
df_acc.to_excel('Wilds_Accessory.xlsx')

sheet theft

In [5]:
import pandas as pd
url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vQwtJ4Y6LJXQ9ORiNE_gTNQuMEQwabLS0lGeKTLPXb4WYHXJwLKDndC-cBl_KCBWQ/pubhtml#"
df = pd.read_html(url, encoding="utf-8")
df = df[0]
df.columns = df.iloc[0]
df = df[1:].reset_index(drop=True)

df['Skill_ID'] = df['skillId']
df['Skill'] = df['skillName']
df['Skill_Type'] = df['skillCategory']
df_skills = df[['Index','Skill_ID','Skill','Skill_Type']].copy()

dict_skills = dict(zip(df_skills['Skill_ID'], df_skills['Skill']))
# df.head()
# df.to_excel("Skill_Names.xlsx")